In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git


import sklearn
import sklearn.feature_extraction.text

#import scipy.cluster.hierarchy
import gensim#For topic modeling
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

In [2]:
df = pandas.read_csv('t\df_10.csv', index_col=0)

In [3]:
#Apply our functions
df['tokenized_text'] = df['AB'].apply(lambda x: lucem_illud_2020.word_tokenize(x))
df['normalized_tokens'] = df['tokenized_text'].apply(lambda x: lucem_illud_2020.normalizeTokens(x))

In [4]:
ngTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5, 
                                                                 max_features=1000, min_df=3, 
                                                                 stop_words='english', 
                                                                 norm='l2')
groupsTFVects = ngTFVectorizer.fit_transform(df['AB'])

In [5]:
def dropMissing(wordLst, vocab):
    return [w for w in wordLst if w in vocab]

df['reduced_tokens'] = df['normalized_tokens'].apply(lambda x: dropMissing(x, ngTFVectorizer.vocabulary_.keys()))

In [6]:
dictionary = gensim.corpora.Dictionary(df['reduced_tokens'])

In [7]:
## create a list of tuples containing each token and its count 
## use the first half and save the remainder for testing 
corpus = [dictionary.doc2bow(text) for text in df['reduced_tokens']]

In [8]:
# serialize the corpus as a file and load it 
gensim.corpora.MmCorpus.serialize('abstract.mm', corpus)
abmm = gensim.corpora.MmCorpus('abstract.mm')

In [9]:
# correctly formatted corpus that can be used for topic modeling and induction 
senlda = gensim.models.ldamodel.LdaModel(corpus=abmm, id2word=dictionary, num_topics=20, alpha='auto', eta='auto')

In [10]:
# sen1Bow = dictionary.doc2bow(df['reduced_tokens'][0])
# sen1lda = senlda[sen1Bow]
# print("The topics of the text: {}".format(df['TI'][0]))
# print("are: {}".format(sen1lda))

In [11]:
ldaDF = pandas.DataFrame({
        'titles' : df['TI'],
        'topics' : [senlda[dictionary.doc2bow(l)] for l in df['reduced_tokens']]
    })

In [12]:
topicsProbDict = {i : [0] * len(ldaDF) for i in range(senlda.num_topics)}

for index, topicTuples in enumerate(ldaDF['topics']):
    for topicNum, prob in topicTuples:
        topicsProbDict[topicNum][index] = prob

for topicNum in range(senlda.num_topics):
    ldaDF['topic_{}'.format(topicNum)] = topicsProbDict[topicNum]

In [13]:
K = senlda.num_topics  # N documents, K topics
topic_labels = ['Topic #{}'.format(k) for k in range(K)]

topicsDict = {}
for topicNum in range(senlda.num_topics):
    topicWords = [w for w, p in senlda.show_topic(topicNum)]
    topicsDict['Topic_{}'.format(topicNum)] = topicWords

wordRanksDF = pandas.DataFrame(topicsDict)
wordRanksDF

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Topic_10,Topic_11,Topic_12,Topic_13,Topic_14,Topic_15,Topic_16,Topic_17,Topic_18,Topic_19
0,factor,factor,need,factor,patient,risk,patient,factor,group,care,hiv,determinant,use,factor,group,life,determinant,determinant,child,family
1,self,level,care,patient,care,result,determinant,determinant,determinant,determinant,determinant,level,factor,determinant,factor,use,use,intervention,care,determinant
2,determinant,risk,factor,risk,method,high,treatment,individual,child,method,group,result,care,year,research,age,high,result,age,risk
3,risk,determinant,child,effect,determinant,education,community,need,intervention,result,result,high,patient,result,method,child,test,review,determinant,result
4,care,high,determinant,child,factor,factor,factor,include,include,low,child,rate,determinant,analysis,sexual,year,analysis,physical,year,group
5,include,individual,mental,determinant,result,determinant,disease,community,ci,patient,low,factor,analysis,method,determinant,high,sleep,method,include,child
6,result,result,result,age,analysis,cancer,result,result,result,research,age,child,risk,group,result,determinant,patient,activity,result,factor
7,policy,community,analysis,level,high,method,family,service,factor,conclusion,disease,method,cancer,intention,care,result,self,community,factor,intervention
8,group,age,practice,high,exercise,age,model,high,cancer,service,method,mortality,ci,care,year,group,service,behavior,analysis,disease
9,method,time,service,mental,model,ci,care,rate,low,population,rural,disease,model,age,suicide,factor,factor,participation,program,life
